In [1]:
import $ivy.`org.flinkextended::flink-scala-api:1.17.1_1.1.0`

import $ivy.`org.apache.flink:flink-clients:1.17.1`
import $ivy.`org.apache.flink:flink-table-api-java:1.17.1`
import $ivy.`org.apache.flink:flink-table-api-java-bridge:1.17.1`
import $ivy.`org.apache.flink:flink-table-runtime:1.17.1`
import $ivy.`org.apache.flink:flink-table-planner-loader:1.17.1`
import $ivy.`org.apache.flink:flink-runtime-web:1.17.1`

import $ivy.`org.apache.iceberg:iceberg-flink-runtime-1.17:1.4.2`

import $ivy.`org.apache.hadoop:hadoop-common:2.7.3`
import $ivy.`org.apache.hadoop:hadoop-hdfs:2.7.3`
import $ivy.`org.apache.hadoop:hadoop-mapreduce-client-core:2.7.3`

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$

In [8]:
import org.apache.iceberg.flink.actions.Actions
import org.apache.iceberg.flink.TableLoader
import org.apache.flink.configuration.{Configuration, ConfigConstants}
import org.apache.flink.configuration.RestOptions.BIND_PORT
import org.apache.flinkx.api._
import scala.collection.JavaConverters._

val config = Configuration.fromMap(
  Map(
    ConfigConstants.LOCAL_START_WEBSERVER -> "true",
    BIND_PORT.key -> "8081",
    "execution.checkpointing.interval" -> "10 s"
  ).asJava
)

val env = StreamExecutionEnvironment.createLocalEnvironmentWithWebUI(config)
val tableLoader = TableLoader.fromHadoopTable("file:///tmp/iceberg/default/sales_country")
tableLoader.open()
val table = tableLoader.loadTable()

1 deprecation
1 deprecation (since 2.13.0)
2 deprecations in total; re-run with -deprecation for details


import org.apache.iceberg.flink.actions.Actions
import org.apache.iceberg.flink.TableLoader
import org.apache.flink.configuration.{Configuration, ConfigConstants}
import org.apache.flink.configuration.RestOptions.BIND_PORT
import org.apache.flinkx.api._
import scala.collection.JavaConverters._
config: Configuration = {execution.checkpointing.interval=10 s, rest.port=8081, rest.bind-port=8081, local.start-webserver=true}
env: StreamExecutionEnvironment = org.apache.flinkx.api.StreamExecutionEnvironment@112e1cb4
tableLoader: TableLoader = HadoopTableLoader{location=file:///tmp/iceberg/default/sales_country}
table: org.apache.iceberg.Table = file:///tmp/iceberg/default/sales_country

In [9]:
val res = Actions.forTable(env.getJavaEnv, table)
	.rewriteDataFiles()
	.maxParallelism(5)
	.targetSizeInBytes(10*1024*1024)
	.execute()

res: org.apache.iceberg.actions.RewriteDataFilesActionResult = org.apache.iceberg.actions.RewriteDataFilesActionResult@3940adcd

In [10]:
res.deletedDataFiles.asScala

1 deprecation (since 2.13.0); re-run with -deprecation for details


res10: collection.mutable.Buffer[org.apache.iceberg.DataFile] = Buffer(
  GenericDataFile{content=data, file_path=file:/tmp/iceberg/default/sales_country/data/00000-0-10f4c905-d73e-421c-a8e5-e8e76f85b1fe-00006.parquet, file_format=PARQUET, spec_id=0, partition=PartitionData{}, record_count=9, file_size_in_bytes=821, column_sizes=null, value_counts=null, null_value_counts=null, nan_value_counts=null, lower_bounds=null, upper_bounds=null, key_metadata=null, split_offsets=[4], equality_ids=null, sort_order_id=null, data_sequence_number=3, file_sequence_number=3},
  GenericDataFile{content=data, file_path=file:/tmp/iceberg/default/sales_country/data/00000-0-10f4c905-d73e-421c-a8e5-e8e76f85b1fe-00012.parquet, file_format=PARQUET, spec_id=0, partition=PartitionData{}, record_count=8, file_size_in_bytes=819, column_sizes=null, value_counts=null, null_value_counts=null, nan_value_counts=null, lower_bounds=null, upper_bounds=null, key_metadata=null, split_offsets=[4], equality_ids=null, sort_or

In [11]:
res.addedDataFiles.asScala

1 deprecation (since 2.13.0); re-run with -deprecation for details


res11: collection.mutable.Buffer[org.apache.iceberg.DataFile] = Buffer(
  GenericDataFile{content=data, file_path=file:/tmp/iceberg/default/sales_country/data/00001-0-a569c932-0da1-4a41-b500-5431640cad2d-00006.parquet, file_format=PARQUET, spec_id=0, partition=PartitionData{}, record_count=1, file_size_in_bytes=679, column_sizes={1=41, 2=42}, value_counts={1=1, 2=1}, null_value_counts={1=0, 2=0}, nan_value_counts={}, lower_bounds=org.apache.iceberg.SerializableByteBufferMap@838e5d6, upper_bounds=org.apache.iceberg.SerializableByteBufferMap@838e5d6, key_metadata=null, split_offsets=[4], equality_ids=null, sort_order_id=0, data_sequence_number=null, file_sequence_number=null},
  GenericDataFile{content=data, file_path=file:/tmp/iceberg/default/sales_country/data/00000-0-bd047a9a-c18e-4159-8e67-485ff4181c64-00007.parquet, file_format=PARQUET, spec_id=0, partition=PartitionData{}, record_count=2, file_size_in_bytes=725, column_sizes={1=64, 2=46}, value_counts={1=2, 2=2}, null_value_counts=

In [ ]:
env.getJavaEnv.close

In [39]:
import scala.concurrent.duration._

val tsToExpire = System.currentTimeMillis() - 10.minutes.toMillis
println(new java.util.Date(tsToExpire))

table.expireSnapshots()
     .expireOlderThan(tsToExpire)
     .commit()

Tue Nov 28 13:05:49 CET 2023


import scala.concurrent.duration._
tsToExpire: Long = 1701173149985L